In [5]:
# Install necessary packages if not already installed
# !pip install torch torchvision opacus wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


Using device: cuda


In [15]:
print(wandb.__path__) 

_NamespacePath(['/home/da23c014/PrivacyAI/CourseProject/dataset_reconstruction/wandb'])


In [12]:
# Initialize wandb
import wandb

wandb_project_name = 'FER2013_DPSGD_Binary_Classification'
wandb_run_name = 'MLP_d-1000-1000-1_Epsilon120'

# Login to wandb (uncomment if necessary)
# !wandb login

# Initialize wandb run
wandb.init(project=wandb_project_name)


AttributeError: module 'wandb' has no attribute 'init'

In [3]:
# Custom Dataset for loading data from CSV files
class FER2013Dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform if transform is not None else transforms.ToTensor()
        
        # Filter data to include only labels 3 and 4 (Happy and Fear)
        self.data = self.data[self.data['emotion'].isin([3, 4])].reset_index(drop=True)
        
        # Convert labels to binary labels: 0 (Happy) and 1 (Fear)
        self.label_mapping = {3: 0, 4: 1}
        self.data['emotion'] = self.data['emotion'].map(self.label_mapping)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        label = int(self.data.iloc[idx]['emotion'])
        pixels = np.array(self.data.iloc[idx]['pixels'].split(), dtype=np.float32).reshape(48, 48)
        
        if self.transform:
            pixels = self.transform(pixels)
        
        return pixels, label


In [4]:
# Paths to the training and test CSV files
train_csv = '/home/da23c014/PrivacyAI/CourseProject/datasets/fer2013/train.csv'  # Replace with the actual path
test_csv = '/home/da23c014/PrivacyAI/CourseProject/datasets/fer2013/test.csv'    # Replace with the actual path

# Define transformations (if any)
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add any additional transformations if needed
])

# Create datasets
train_dataset = FER2013Dataset(csv_file=train_csv, transform=transform)
test_dataset = FER2013Dataset(csv_file=test_csv, transform=transform)

# Define batch size
batch_size = 4096  # Be cautious with memory usage

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f'Train dataset size: {len(train_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')


Train dataset size: 7829
Test dataset size: 4216
